### To do


> Two datasets "MET-Meme & Memotion" (Multimodal image and text) offensive Meme detection 

> "Multiclass: Not offensive, slight offensive and very offensive" 

> Two Models Models are parallelly work: 
    on text (BERT and CNN) on images (RESNet-152, also comment ResNet-50) 
    Then Dense layer, fully concatenation layer and final multi classification (softmax). 
    Requirement is Dataset should be 7k or more also balance dataset. 
    Accuracy, recall, f1score, precision (validation and training loss graph), 
    (validation and training accuracy graph), (Confusion Matrix)

**Libraries**

In [ ]:
rm -rf *pkl

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from PIL import Image
import cv2 
import seaborn as sns
import matplotlib.pyplot as plt

import pickle
import os

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.models import resnet152
from transformers import BertModel, BertTokenizer

## EDA and Preprocessing

### *Dataset Path*

In [ ]:
ROOT_PATH = "/kaggle/input/memotion-dataset-7k/memotion_dataset_7k"
ROOT_PATH2 = "/kaggle/input/met-meme"

In [ ]:
data1 = pd.read_csv(f"{ROOT_PATH}/labels.csv")
# data.drop(columns= ["Unnamed: 0", "text_ocr", "humour", "sarcasm", "motivational", "overall_sentiment"],
#             inplace = True)
# data.columns = ["image_name", "text", "label"]


data1.drop(columns= ["Unnamed: 0","text_ocr", 'humour', 'sarcasm', 'motivational' , 'overall_sentiment'], inplace = True)
data1.columns = ["image_name", "text", "label"]

data1.head()

In [ ]:
labelDict = {'1(slightly)': 'slight', '0(non-offensive)':'not_offensive', '3(very)':'hateful_offensive', '2(moderately)':'very_offensive'}

In [ ]:
data2 = pd.read_csv(f"{ROOT_PATH2}/E_text.csv", encoding = 'latin-1')
labelsDF = pd.read_csv(f"{ROOT_PATH2}/label_E.csv", encoding = 'latin-1')
labelsDF['offensiveness detection'] = labelsDF['offensiveness detection'].map(labelDict)
labelsDF.drop(columns=["sentiment category","sentiment degree", "intention detection", "metaphor occurrence", 
                       "metaphor category", "target domain", "source domain", "target modality", "source modality"], inplace=True)
labelsDF.columns = ['file_name', 'label']

data2.head()

In [ ]:
labelsDF.head()

In [ ]:
dataset2 = pd.merge(data2, labelsDF, on='file_name')
dataset2.columns = ['image_name', 'text', 'label']
dataset2

In [ ]:
data = pd.concat([data1, dataset2])
data.head()

In [ ]:
import os
import shutil

NEW_ROOT_PATH = "/kaggle/working/data_images"

if os.path.exists(NEW_ROOT_PATH):
    # Use os.rmdir() to remove an empty directory
    shutil.rmtree(NEW_ROOT_PATH)
    print("Directory removed successfully.")

os.mkdir(NEW_ROOT_PATH)

source_folder1 = "/kaggle/input/memotion-dataset-7k/memotion_dataset_7k/images"
source_folder2 = "/kaggle/input/met-meme/Eimages/Eimages/Eimages"

def copy_images(source_folder, destination_folder):
    for filename in os.listdir(source_folder):
        shutil.copy(os.path.join(source_folder, filename), destination_folder)

# Copy images from source_folder1
copy_images(source_folder1, NEW_ROOT_PATH)

# Copy images from source_f`older2
copy_images(source_folder2, NEW_ROOT_PATH)



In [ ]:
# os.listdir('/kaggle/working/data_images')

### Plot Distribution
> Plotting the distribution of Labels

In [ ]:
def plot_distribution(df, column_name):
    
    Distribution = df[column_name].value_counts().to_dict()
    categories = list(Distribution.keys())
    values = list(Distribution.values())

    plt.figure(figsize=(10, 6))
    sns.barplot(x=categories, y=values, palette='Set2')
    for i, v in enumerate(values):
        plt.text(i, v + 50, str(v), ha='center', va='bottom')
    plt.xlabel('Offensiveness Categories')
    plt.ylabel('Frequency')
    plt.title('Frequency of Offensiveness Categories')
    plt.xticks(rotation=45)  # Rotating x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    plt.show()



In [ ]:
plot_distribution(data, "label")

### Balancing the Dataset

> merged "very_offensive" & "hateful_offensive" as 'very_offensive'

In [ ]:
import pandas as pd

# Merge 'very_offensive' and 'hateful_offensive' into one category
data['label'] = data['label'].replace({ 'hateful_offensive': 'very_offensive'})
plot_distribution(data, "label")


### Splitting Data for Training, Validating and Testing

Ensuring Equal Distribution of Labels in Each Subset

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

X = data.drop(columns = ["label"])
y = data["label"]

# Step 1: Shuffle the dataset
indices = np.arange(len(X))
np.random.shuffle(indices)
X_shuffled = X.iloc[indices]
y_shuffled = y.iloc[indices]

# Step 2: Calculate the percentage of each label
unique_labels, label_counts = np.unique(y_shuffled, return_counts=True)
label_percentages = label_counts / len(y_shuffled)

# Step 3: Split the dataset while maintaining consistent label percentages
# You can adjust the test_size and validation_size as needed
X_train, X_temp, y_train, y_temp = train_test_split(X_shuffled, y_shuffled, test_size=0.2, stratify=y_shuffled)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

X_train.reset_index(drop=True, inplace = True)
y_train.reset_index(drop=True, inplace = True)

X_validation.reset_index(drop=True, inplace = True)
y_validation.reset_index(drop=True, inplace = True)

X_test.reset_index(drop=True, inplace = True)
y_test.reset_index(drop=True, inplace = True)

# Check label percentages in each set
train_label_percentage = np.unique(y_train, return_counts=True)[1] / len(y_train)
validation_label_percentage = np.unique(y_validation, return_counts=True)[1] / len(y_validation)
test_label_percentage = np.unique(y_test, return_counts=True)[1] / len(y_test)

print("Label percentages in training set:", train_label_percentage)
print("Label percentages in validation set:", validation_label_percentage)
print("Label percentages in test set:", test_label_percentage)


In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_validation, y_validation], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)


In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
val_df.info()

### One Hot Encoding the lables

In [ ]:
# Perform one-hot encoding
train_df = pd.get_dummies(train_df, columns=['label'], prefix = "", 
                            prefix_sep = "",dtype = "float32")

val_df = pd.get_dummies(val_df, columns=['label'], prefix = "", 
                            prefix_sep = "",dtype = "float32")

test_df = pd.get_dummies(test_df, columns=['label'], prefix = "", 
                            prefix_sep = "",dtype = "float32")


### Label Encoding

In [ ]:
label_encoder = LabelEncoder()

# # Apply LabelEncoder to each column
# for column in ["humour", "sarcasm", "motivational", "overall_sentiment"]:
#     train_df[column] = label_encoder.fit_transform(train_df[column])
#     val_df[column] = label_encoder.fit_transform(val_df[column])
#     test_df[column] = label_encoder.fit_transform(test_df[column])
    

In [ ]:
train_df.head()

## Making Dataset 

In [ ]:
class MemotionDataset(Dataset):
    def __init__(self, df: pd.DataFrame, image_path: str, tokenizer, transforms, 
                 device :str = "cpu",mode :str = "train", pickle_name :str = "data.pkl"):
        super().__init__()
        self.df = df
        self.imgPath = image_path
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.device = device
        self.image_data = []
        self.pickle_file = mode + "_" + pickle_name

#         If pickle file doesn't exist, preprocess images and save to pickle file
        if not os.path.exists(self.pickle_file):
            self.preprocess_images()
        else:
            with open(self.pickle_file, 'rb') as f:
                self.image_data = pickle.load(f)

    def preprocess_images(self):
        for ix, row in self.df.iterrows():
            image_path = os.path.join(self.imgPath, row['image_name'])
            try:
                img = np.array(Image.open(image_path).convert('RGB'))
                img = self.transforms(img)
                text = str(row['text']).lower()
                out = self.tokenizer(
                    text=text,
                    max_length=124,
                    padding='max_length',
                    truncation=True,
                    return_tensors="pt"
                )
#                 humour, sarcasm, motivational, overall_sentiment = row[["humour", "sarcasm", "motivational", "overall_sentiment"]]
                label = torch.tensor([row['not_offensive'], row['slight'], row['very_offensive']], dtype=torch.float32)
                self.image_data.append({
                    'image': img,
                    'input_ids': torch.concat([out['input_ids'].squeeze()]), 
                    'attention_mask': torch.concat([out['attention_mask'].squeeze()]),
                    'label': label,
                })
            except:
                continue  # Skip

        # Save image data to pickle file
        with open(self.pickle_file, 'wb') as f:
            pickle.dump(self.image_data, f)

    def __len__(self) -> int:
        return len(self.image_data)

    def __getitem__(self, ix: int) -> dict[str, torch.Tensor]:
        data = self.image_data[ix]
        return {
            'image': data['image'].to(self.device),
            'input_ids': data['input_ids'].to(self.device),
            'attention_mask': data['attention_mask'].to(self.device),
            'label': data['label'].to(self.device),
        }


In [ ]:
#         # Text model (BERT)
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.bert_out_features = bert_hidden_size

#         # Image model (ResNet-152)
#         self.resnet = resnet152(pretrained=True)
#         self.resnet.fc = nn.Identity()  # We remove the final fully connected layer

#         # Attention mechanism
#         self.attention_fc = nn.Linear(bert_hidden_size + 2048, 1)

#         # Final classification layer
#         self.fc_layers = nn.Sequential(
#             nn.Linear(bert_hidden_size + 2048, 128),
#             nn.LayerNorm(128),
#             nn.ReLU(),
#             nn.Linear(128, num_classes),
#         )

#     def forward(self, input_ids, attention_mask, image):
#         bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]  # Taking pooled output
#         resnet_output = self.resnet(image)
#         print("bert:", bert_output.size())
#         print('resent:', resnet_output.size())

#         # Concatenate BERT and ResNet outputs
#         combined_features = torch.cat((bert_output, resnet_output), dim=1)
#         print('comb:', combined_features.size())

#         # Calculate attention scores
#         attention_scores = torch.tanh(self.attention_fc(combined_features)).squeeze(-1)
#         attention_weights = F.softmax(attention_scores, dim=0)
        
#         print('attn:',attention_weights.unsqueeze(0).size())
#         # Apply attention weights to the ResNet features
#         weighted_resnet_features = torch.bmm(attention_weights,resnet_output).squeeze(1)
#         print('weight attn',weighted_resnet_features.size())

#         # Concatenate attention-weighted ResNet features with BERT features
#         fused_features = torch.cat((bert_output, weighted_resnet_features), dim=1)

#         # Classification layer
#         output = self.fc_layers(fused_features)
#         output = F.softmax(output, dim=1)

#         return output  

In [ ]:
# # # Define multimodal model
# # class MultiModalModel(nn.Module):
# #     def __init__(self, num_classes, bert_hidden_size=768):
# #         super(MultiModalModel, self).__init__()  
# #         # Text model (BERT)
# #         self.bert = BertModel.from_pretrained('bert-base-uncased')
# #         self.bert_out_features = self.bert.config.hidden_size
        
# #         # Image model (ResNet-152)
# #         self.resnet = resnet152(pretrained=True)
        
# #         # Change the output layer to match the BERT output size
# #         self.resnet.fc = nn.Linear(self.resnet.fc.in_features, self.bert_out_features)
        
# #         # Final classification layer
# #         self.fc_layers = nn.Sequential(
# #             nn.Linear(self.bert_out_features * 2, 128),  # Concatenating BERT and ResNet outputs
# #             nn.LayerNorm(128),
# #             nn.ReLU(),
# #             nn.Linear(128, 64),
# #             nn.LayerNorm(64),
# #             nn.ReLU(),
# #             nn.Linear(64, num_classes),
# #         )

# #     def forward(self, input_ids, attention_mask, image):
# #         bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]  # Taking pooled output
# #         resnet_output = self.resnet(image)
        
# #         # Concatenate BERT and ResNet outputs
# #         combined_features = torch.cat((bert_output, resnet_output), dim=1)
        
# #         # Classification layer
# #         output = self.fc_layers(combined_features)
# #         output = F.softmax(output, dim=1)
# #         return output



# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from transformers import BertModel
# from torchvision.models import vgg16

# # Define multimodal model
# class MultiModalModel(nn.Module):
#     def __init__(self, num_classes, bert_hidden_size=768):
#         super(MultiModalModel, self).__init__()  
#         # Text model (BERT)
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.bert_out_features = self.bert.config.hidden_size
        
#         # Image model (VGG-16)
#         self.vgg16 = vgg16(pretrained=True)
#         # Replace the final layer to match the BERT output size
#         self.vgg16.classifier[-1] = nn.Linear(self.vgg16.classifier[-1].in_features, self.bert_out_features)
        
#         # Final classification layer
#         self.fc_layers = nn.Sequential(
#             nn.Linear(self.bert_out_features * 2, 128),  # Concatenating BERT and VGG outputs
#             nn.LayerNorm(128),
#             nn.ReLU(),
#             nn.Linear(128, 64),
#             nn.LayerNorm(64),
#             nn.ReLU(),
#             nn.Linear(64, num_classes),
#         )

#     def forward(self, input_ids, attention_mask, image):
#         bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]  # Taking pooled output
#         vgg_output = self.vgg16(image)
        
#         # Concatenate BERT and VGG outputs
#         combined_features = torch.cat((bert_output, vgg_output), dim=1)
        
#         # Classification layer
#         output = self.fc_layers(combined_features)
#         output = F.softmax(output, dim=1)
#         return output

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from torchvision.models import vgg16

class MultiModalModel(nn.Module):
    def __init__(self, num_classes, bert_hidden_size=768, vgg_output_size=1000):
        super(MultiModalModel, self).__init__()  
        # Text model (BERT)
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert_out_features = self.bert.config.hidden_size
        
        # Image model (VGG-16)
        self.vgg16 = vgg16(pretrained=True)
        self.vgg_out_features = vgg_output_size
        
        # Attention mechanisms
        self.text_att_linear = nn.Linear(self.bert_out_features, self.vgg_out_features)
        self.image_att_linear = nn.Linear(self.vgg_out_features, self.bert_out_features)
        
        # Intermediate layers to adjust sizes
        self.text_adjust = nn.Linear(self.vgg_out_features, self.bert_out_features)
        self.image_adjust = nn.Linear(self.bert_out_features, self.vgg_out_features)
        
        # Final classification layer
        self.fc_layers = nn.Sequential(
            nn.Linear(self.bert_out_features + self.vgg_out_features, 128),  # Adjust input size
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, input_ids, attention_mask, image):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]  # Taking pooled output
        vgg_output = self.vgg16(image)
        
        # Attention mechanisms
        text_att_weights = F.softmax(self.text_att_linear(bert_output), dim=1)
        image_att_weights = F.softmax(self.image_att_linear(vgg_output), dim=1)
        
        # Adjust sizes of attention weights
        text_att_weights_adjusted = self.text_adjust(text_att_weights)
        image_att_weights_adjusted = self.image_adjust(image_att_weights)
        
        # Multiply original features with attention weights
        attended_text_features = torch.mul(bert_output, text_att_weights_adjusted)
        attended_image_features = torch.mul(vgg_output, image_att_weights_adjusted)
        
        # Concatenate attended features
        combined_features = torch.cat((attended_text_features, attended_image_features), dim=1)
        
        # Classification layer
        output = self.fc_layers(combined_features)
        output = F.softmax(output, dim=1)
        return output


In [ ]:

# Training function
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        train_loader_iter = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in train_loader_iter:
            inputs, attention, images, labels = batch['input_ids'], batch['attention_mask'], batch['image'], batch['label']
            optimizer.zero_grad()
            outputs = model(inputs, attention, images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.argmax(labels, 1)).sum().item()
            train_loader_iter.set_postfix({'loss': running_loss / total})
        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_accuracy)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        val_loader_iter = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
        with torch.no_grad():
            for batch in val_loader_iter:
                inputs, attention, images, labels = batch['input_ids'], batch['attention_mask'], batch['image'], batch['label']
                outputs = model(inputs, attention, images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == torch.argmax(labels, 1)).sum().item()
                val_loader_iter.set_postfix({'loss': loss.item()})
        val_losses.append(val_loss / len(val_loader))
        val_accuracy = correct / total
        val_accs.append(val_accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.4f}")

    return train_losses, val_losses, train_accs, val_accs


In [ ]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Data preparation
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
IMAGE_PATH = NEW_ROOT_PATH + '/'
# os.path.join(NEW_ROOT_PATH, "/")
trainSet = MemotionDataset(train_df,IMAGE_PATH, tokenizer, transform, device,mode = "train")
valSet = MemotionDataset(val_df,IMAGE_PATH, tokenizer, transform, device, mode = "val")
testSet = MemotionDataset(test_df,IMAGE_PATH, tokenizer, transform, device, mode = "test")

# Hyperparameters
num_classes = 3
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# Initialize datasets and loaders
train_loader = DataLoader(trainSet, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valSet, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testSet, batch_size=batch_size, shuffle=False)

In [ ]:
# os.listdir('/kaggle/working/data_images/')


In [ ]:
# Initialize model
model = MultiModalModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# model

In [ ]:
batch = next(iter(train_loader))
inputs, attention, images, labels = batch['input_ids'], batch['attention_mask'], batch['image'], batch['label']

In [ ]:
# Train the model
train_losses, val_losses, train_accs, val_accs = train(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

# Plotting training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_accs, label='Train Acc')
plt.plot(range(1, num_epochs+1), val_accs, label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
import json
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoModel, AutoTokenizer, get_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW, SGD
from tqdm.notebook import tqdm, trange
from time import perf_counter
from PIL import Image
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
device

In [ ]:
!pip install wget

In [ ]:
import os
ALBEF_FOLDER = 'ALBEF/'
os.makedirs(ALBEF_FOLDER, exist_ok=True)

In [ ]:
# download pre-trained ALBEF model and required ALBEF files from ALBEF's official repo (only need to do this once to save it in your gdrive)
!wget https://raw.githubusercontent.com/salesforce/ALBEF/main/models/vit.py -O $ALBEF_FOLDER/vit.py
!wget https://raw.githubusercontent.com/salesforce/ALBEF/main/models/tokenization_bert.py -O $ALBEF_FOLDER/tokenization_bert.py
!wget https://raw.githubusercontent.com/salesforce/ALBEF/main/models/xbert.py -O $ALBEF_FOLDER/xbert.py


In [ ]:
# replace all occurrences of tokenizer_class with processor_class in xbert.py to make it compatible with newer transformers version
# if you don't do this step, you will need to install transformers==4.8.1 as specified by the requirements in the ALBEF repo

!sed -i 's/tokenizer_class/processor_class/g' $ALBEF_FOLDER/xbert.py

In [ ]:
# add path to downloaded ALBEF files
import sys
sys.path.append(ALBEF_FOLDER)

#import libraries required for ALBEF
from vit import VisionTransformer
from xbert import BertConfig as AlbefBertConfig, BertModel as AlbefBertModel
from functools import partial
import numpy as np

In [ ]:
data = data.dropna(axis=0)

In [ ]:
import pickle
def load_all_images_and_text(df, path):
    images_paths = df['image_name'].to_list()
    text = df['text'].to_list()
    texts = []
    label = df['label'].to_list()
    labels = []
    images = []
    for ind, filename in enumerate(images_paths):
        try:
            img = Image.open(os.path.join(path,filename)).convert('RGB')
        except:
            continue
        images.append(img)
        texts.append(text[ind])
        labels.append(label[ind])

    return texts, images, labels

text, images, labels = load_all_images_and_text(data, path = NEW_ROOT_PATH)

with open("text", "wb") as fp:
    pickle.dump(text, fp)

with open("images", "wb") as fp:
    pickle.dump(images, fp)

with open("labels", "wb") as fp:
    pickle.dump(labels, fp)

In [ ]:
len(images)

In [ ]:
len(text)

In [ ]:
len(labels)

In [ ]:
class AlbefDataset(Dataset):
    def __init__(self, text, images, label):
        self.text = text
        self.images = images
        self.label = label

        # ALBEF settings
        self.img_size = 256
        self.mean, self.std = (
            0.40821073), (0.26862954)


        self.train_transform_func = transforms.Compose(
                [transforms.RandomResizedCrop(self.img_size, scale=(0.5, 1.0)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(self.mean, self.std)
                    ])

                    
    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.label[idx]
        if label == 'not_offensive':
            label = 0
        elif label == 'very_offensive':
            label = 1
        else:
            label = 2
        image = self.images[idx]
        
        
        img = self.train_transform_func(image)

        return text, img, label

    def __len__(self):
        return len(self.text)

dataset = AlbefDataset(text, images, labels)
dataset[2]

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
class AlbefModel(nn.Module):

    def __init__(self, bert_config, num_labels, text_pretrained='bert-base-uncased'):
        super().__init__()

        self.num_labels = num_labels
        self.text_encoder = AlbefBertModel.from_pretrained(
            text_pretrained, config=bert_config, add_pooling_layer=False)

        self.visual_encoder = VisionTransformer(
            img_size=256, patch_size=16, embed_dim=768, depth=12, num_heads=12,
            mlp_ratio=4, qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6))

        self.classifier = nn.Linear(
            self.text_encoder.config.hidden_size, num_labels)
        
    
    def forward(self, text, image):
        image_embeds = self.visual_encoder(image)
        image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image_embeds.device)
        output = self.text_encoder(text.input_ids, attention_mask=text.attention_mask,
                                   encoder_hidden_states=image_embeds, encoder_attention_mask=image_atts, return_dict=True
                                   )
        logits = self.classifier(output.last_hidden_state[:, 0, :])
        return logits

In [ ]:
from urllib.request import urlretrieve

def load_albef_pretrained(num_out_labels):
    tmp_directory = './tmp/albef'
    os.makedirs(tmp_directory, exist_ok=True)

    albef_bert_config_fp = os.path.join(tmp_directory, 'config_bert.json')
    albef_model_fp = os.path.join(tmp_directory, 'ALBEF.pth')

    if not os.path.exists(albef_bert_config_fp):
        urlretrieve("https://raw.githubusercontent.com/salesforce/ALBEF/main/configs/config_bert.json", albef_bert_config_fp)

    if not os.path.exists(albef_model_fp):
        urlretrieve("https://storage.googleapis.com/sfr-pcl-data-research/ALBEF/ALBEF_4M.pth", albef_model_fp)

    albef_bert_config = AlbefBertConfig.from_json_file(albef_bert_config_fp)
    albef_model = AlbefModel(bert_config=albef_bert_config, num_labels=num_out_labels)

    print(albef_model_fp)
    albef_checkpoint = torch.load(albef_model_fp, map_location='cpu')
    albef_state_dict = albef_checkpoint['model']

    for key in list(albef_state_dict.keys()):
        if 'bert' in key:
            encoder_key = key.replace('bert.', '')
            albef_state_dict[encoder_key] = albef_state_dict[key]
            del albef_state_dict[key]

    msg = albef_model.load_state_dict(albef_state_dict, strict=False)
    print("ALBEF checkpoint loaded from ", albef_model_fp)
    # print(msg)
    return albef_model

In [ ]:
albef_model = load_albef_pretrained(num_out_labels=1)

In [ ]:
albef_model.parameters()

In [ ]:
from transformers import AutoImageProcessor
import numpy as np

b_inputs = bert_tokenizer(
            "not hello", truncation=True, max_length=512,
            return_tensors="pt", padding='max_length'
        ).to(device)

# b_inputs = b_inputs.to(device)
b_imgs = Image.open(NEW_ROOT_PATH + "/image_445.jpg")
b_imgs = b_imgs.resize((256,256))
b_imgs = torch.Tensor(np.array(b_imgs)).unsqueeze(0).permute(0, 3, 1, 2).to(device)
b_inputs, b_imgs.shape
b_imgs = b_imgs.to(device)

In [ ]:
# x = albef_model(b_inputs, b_imgs)
# x

In [ ]:
# import time

# # Start timer
# start_time = time.perf_counter()
# x = albef_model(b_inputs, b_imgs)
# x.mean().backward()

# end_time = time.perf_counter()

# # Calculate elapsed time
# elapsed_time = end_time - start_time
# print("Elapsed time: ", elapsed_time)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True)
train_data_loader

In [ ]:
from torchvision.ops import sigmoid_focal_loss
num_train_epochs = 5
learning_rate = 0.0005
t_total = len(train_data_loader) * num_train_epochs


optimizer = SGD(albef_model.parameters(), lr=learning_rate)
# scheduler = get_scheduler(name="cosine", optimizer=optimizer, num_warmup_steps=0, num_training_steps=t_total)

criterion = sigmoid_focal_loss

In [ ]:
torch.cuda.empty_cache()
albef_model = albef_model.to(device)
albef_model.train()

for epoch_num in trange(num_train_epochs, desc='Epochs'):
    epoch_total_loss = 0

    for step, batch in tqdm(enumerate(train_data_loader), total=len(train_data_loader), desc='Batch'):        
        b_text, b_imgs, b_labels, = batch  
        # print(b_text, b_imgs, b_labels)
        # break
        b_inputs = bert_tokenizer(
            list(b_text), truncation=True, max_length=128,
            return_tensors="pt", padding='max_length'
        )
        
        b_labels = b_labels.to(device)
        b_imgs = b_imgs.to(device)
        b_inputs = b_inputs.to(device)

        albef_model.zero_grad()        
        b_logits = albef_model(text=b_inputs, image=b_imgs).squeeze(1)
        
        loss = criterion(b_logits.to(torch.float32), b_labels.to(torch.float32), reduction = 'mean')

        epoch_total_loss += loss.item()

        # Perform a backward pass to calculate the gradients
        loss.backward()


        optimizer.step()
        # scheduler.step()
        
    avg_loss = epoch_total_loss/len(train_data_loader)


    print('epoch =', epoch_num)
    print('    epoch_loss =', epoch_total_loss)
    print('    avg_epoch_loss =', avg_loss)
    print('    learning rate =', optimizer.param_groups[0]["lr"])

In [ ]:
b_text

In [ ]:
batch